In [3]:
""" Run CPG """
import time
import numpy as np
import matplotlib

# adapt as needed for your system
# from sys import platform
# if platform =="darwin":
#   matplotlib.use("Qt5Agg")
# else:
#   matplotlib.use('TkAgg')

from matplotlib import pyplot as plt

from env.hopf_network import HopfNetwork
from env.quadruped_gym_env import QuadrupedGymEnv


ADD_CARTESIAN_PD = True
TIME_STEP = 0.001
foot_y = 0.0838 # this is the hip length
sideSign = np.array([-1, 1, -1, 1]) # get correct hip sign (body right is negative)

env = QuadrupedGymEnv(render=True,              # visualize
                    on_rack=False,              # useful for debugging!
                    isRLGymInterface=False,     # not using RL
                    time_step=TIME_STEP,
                    action_repeat=1,
                    motor_control_mode="TORQUE",
                    add_noise=False,    # start in ideal conditions
                    # record_video=True
                    )

# initialize Hopf Network, supply gait
# cpg = HopfNetwork(time_step=TIME_STEP, gait="TROT", omega_swing=10*2*np.pi, omega_stance=10*2*np.pi) # Gait, can be TROT, WALK, PACE, BOUND, etc.
cpg = HopfNetwork(time_step=TIME_STEP, gait="WALK", omega_swing=18*2*np.pi, omega_stance=15*2*np.pi) # Gait, can be TROT, WALK, PACE, BOUND, etc.

TEST_STEPS = int(10 / (TIME_STEP))
t = np.arange(TEST_STEPS)*TIME_STEP

# [TODO] initialize data structures to save CPG and robot states
feet_pos_log = np.zeros([4, 3, TEST_STEPS])

############## Sample Gains
# joint PD gains
kp=np.array([100,100,100])
kd=np.array([2,2,2])
# Cartesian PD gains
kpCartesian = np.diag([500]*3)
kdCartesian = np.diag([20]*3)

for j in range(TEST_STEPS):
  # initialize torque array to send to motors
  action = np.zeros(12)
  # get desired foot positions from CPG
  xs,zs = cpg.update()
  # [TODO] get current motor angles and velocities for joint PD, see GetMotorAngles(), GetMotorVelocities() in quadruped.py
  q = env.robot.GetMotorAngles()
  dq = env.robot.GetMotorVelocities()

  # loop through desired foot positions and calculate torques
  ts_feet_pos_log = np.zeros([4,3])
  for i in range(4):
    # initialize torques for legi
    tau = np.zeros(3)
    # get desired foot i pos (xi, yi, zi) in leg frame
    leg_xyz = np.array([xs[i],sideSign[i] * foot_y,zs[i]])
    # call inverse kinematics to get corresponding joint angles (see ComputeInverseKinematics() in quadruped.py)
    leg_q = env.robot.ComputeInverseKinematics(i,leg_xyz) # [TODO]
    # Add joint PD contribution to tau for leg i (Equation 4)
    leg_dq = np.zeros(3) # desired joint velocities (see eq4) i.e. q_dot_d
    tau += kp * (leg_q - q[i*3:i*3+3]) + kd * (leg_dq - dq[i*3:i*3+3]) # [TODO] maybe [i*3:i*3+2] ?

    # add Cartesian PD contribution
    if ADD_CARTESIAN_PD:
      # Get current Jacobian and foot position in leg frame (see ComputeJacobianAndPosition() in quadruped.py)
      # [TODO]
      J,foot_pos = env.robot.ComputeJacobianAndPosition(i)
      # Get current foot velocity in leg frame (Equation 2)
      # [TODO]
      foot_vel = np.matmul(J,dq[i*3:i*3+3])
      desired_foot_vel = np.zeros(3)  # v_d in equation 5
      # Calculate torque contribution from Cartesian PD (Equation 5) [Make sure you are using matrix multiplications]
      # [TODO]

      J_t = np.transpose(J)

      tau += np.matmul(J_t, np.matmul(kpCartesian,(leg_xyz - foot_pos)) + np.matmul(kdCartesian, (desired_foot_vel - foot_vel)))


    # Set tau for legi in action vector
    action[3*i:3*i+3] = tau
    ts_feet_pos_log[i,:] = foot_pos
  # send torques to robot and simulate TIME_STEP seconds
  env.step(action)

  # [TODO] save any CPG or robot states
  feet_pos_log[:,:,j] = ts_feet_pos_log




#####################################################
# PLOTS
#####################################################
# example
# fig = plt.figure()
# plt.plot(t,joint_pos[1,:], label='FR thigh')
# plt.legend()
# plt.show()

error: Not connected to physics server.